In [3]:
from src.metrics.ChfF import chrf_corpus

import pandas as pd

In [28]:
df = pd.read_csv('results/bind_ToxiBenchCN-Qwen3-0.6B-Base-seed=42--epoch=09-valid_score=67.2945.csv')

In [29]:
for category in sorted(set(df['category'])):
    cat_df = df[df['category']==category].copy()
    print(category, chrf_corpus(cat_df['pred'].tolist(), cat_df['true'].tolist())['f1'])

emo 78.8075726834124
homo 56.09504184513585
pinyin 52.34272943685219
radical 69.73511737550903
swap 52.38966617300682
trad 92.18336200032617


In [30]:
df

,input,pred,true,category
0,其他💦方是💦🌧️黑，上海这个只是说了实话怎么能算黑,其他地方是地域黑，上海这个只是说了实话怎么能算黑,其他地方是地域黑，上海这个只是说了实话怎么能算黑,emo
1,什么👩用🀂1️⃣📝，这就把我的臭🧦子塞进去,什么女用西一圾，这就把我的臭造子塞进去,什么女用洗衣机，这就把我的臭袜子塞进去,emo
2,异⭐恋属阳，是♨常的，同⭐恋属阴，是👋♨常的,异性恋属阳，是正常的，同性恋属阴，是不正常的,异性恋属阳，是正常的，同性恋属阴，是不正常的,emo
3,那还有假，这些黑醋最喜欢做的事就是⬆害别人，干违法乱📝的事,那还有假，这些黑醋最喜欢做的事就是伤害别人，干违法乱基的事,那还有假，这些黑醋最喜欢做的事就是伤害别人，干违法乱纪的事,emo
4,每个在华黑人都会有3个以上的eg,每个在华黑人都会有3个以上的eg,每个在华黑人都会有3个以上的eg,emo
...,...,...,...,...
1526,什么？同连小日本都跪？,什么？同同小日本都跪？,什么？同连小日本都跪？,trad
1527,什么二婚幸福大部分瞎編,什么二婚幸福大部分瞎眼,什么二婚幸福大部分瞎编,trad
1528,一般会这么想，竟然真有憨批找黑人,一般会这么想，竟然真有憨批找黑人,一般会这么想，竟然真有憨批找黑人,trad
1529,自己找不到女朋友，怪黑人？,自己找不到女朋友，怪黑人？,自己找不到女朋友，怪黑人？,trad
